In [1]:
import numpy as np
from Coursework.ANNModel import model as mod
from Coursework.ANNModel import data as d
from Coursework.PSO import pswarm as ps
from Coursework.PSO import psobehaviour as psh
from Coursework.PSO import history as hs
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

data = d.Data("./Data/1in_tanh.txt")
X = data.get_rows()
y = data.get_output()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=14)

D:\GitProjects\Bio-comp\Coursework\ANNModel\model.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
#train the model
model = mod.ANN()
model.set_training_input(X_train, y_train)
model.add(mod.Layer(4, activation="sigmoid"))
model.add(mod.Layer(1, activation="null"))
model.compile()

model_history = hs.PSOHistory(model)

Model Compiled!


In [3]:
Cubic_config = [58.36144647,  7.57741494,  0.30742191,  0.23694025,  1.13547031, 1.46194965,  0.92816043] # Fitness: 4436
Linear_config = [6.19884153e+01, 6.19329219e+00, 2.82994987e-02, 9.33192720e-01, 9.06362957e-01, 1.09817852e+00, 1.05772006e+00] # Fitness: 552335863302
Sine_config = [59.35248674, 5.71898616, 1.38796748, 1.48949903, 0.76230353, 0.37289024, 0.98682079] # Fitness: 105
Tanh_config = [64.45651991,  6.25238014,  0.57690358,  1.25084245,  0.93161939, 0.55929343,  0.53392325] # Fitness: 20498

In [4]:
pso = ps.PSO(swarm_size=100, num_informants=6, boundary_policy=psh.BoundaryPolicy.RANDOMREINIT, termination_policy=[psh.TerminationPolicy.ITERATIONS], termination_args={'max_iter': 400}, alpha=0.61, beta=1.3, gamma=1.4, delta=1.3, epsilon=0.6)
pso = pso.decode_vec(Tanh_config)

print(pso.swarm_size)

58


In [5]:
pso.set_search_dimensions(model.dimension_vec())
pso.set_fitness_fn(model_history.evaluate_fitness)
fitness = pso.run()
print(fitness.location)
print(fitness.fitness)

[ 2.68894268  0.65068997  0.89662241  0.50382234  0.00983154  0.36714624
  0.46303986 -0.60690874 -0.54237318  3.0795269  -0.87284109 -0.72488437
 -0.85557772  0.55538627 -0.0838537 ]
33.93084703722546


In [6]:
pso_best = pso.best.location
model.decode_vec(pso_best)

In [7]:
#model.layers[-1].bias = np.array([0.0])
#model.layers[-1].weights = np.array([[1.0]])


In [8]:
#run model
model.set_training_input(X_test, y_test)
model.one_pass()


In [9]:
model.y_hat

array([-0.32489508, -0.14791395, -0.19661441, -0.08591804,  0.0530574 ,
        0.24186422,  0.39746913, -0.13560449,  0.49745678,  0.17957877,
       -0.54658017,  0.48672534, -0.39049052,  0.27870063,  0.55965605,
        0.22948826,  0.01496694, -0.4914935 , -0.26774356, -0.51033402,
       -0.20864085,  0.16701534, -0.16017258, -0.36901289, -0.01038039,
        0.46496628, -0.50097274,  0.29087251,  0.04035563, -0.11084495,
       -0.33606477,  0.51861395,  0.09115367,  0.02765773])

In [10]:
#score function
correctly_classified = 0


#get predictions
predictions = np.round(model.y_hat, 4)
print(model.y)
print(predictions)

#loop through predictions
for index, pred in enumerate(predictions):

    if np.isclose(y_test[index], pred):
        correctly_classified += 1

print('A,B: ', correctly_classified , " out of ", len(y_test))
correctly_classified = 0
#loop through predictions
for index, pred in enumerate(predictions):
    if np.isclose(pred, y_test[index]):
        correctly_classified += 1

print('B,A: ',correctly_classified , " out of ", len(y_test))

[-1.406e-01 -1.060e-02 -2.700e-02 -1.700e-03  1.000e-03  6.400e-02
  2.875e-01 -8.000e-03  5.927e-01  2.700e-02 -8.847e-01  5.514e-01
 -2.621e-01  9.730e-02  8.847e-01  5.490e-02  1.000e-04 -5.927e-01
 -7.410e-02 -6.815e-01 -3.280e-02  2.200e-02 -1.380e-02 -2.160e-01
  0.000e+00  4.746e-01 -6.361e-01  1.106e-01  5.000e-04 -4.100e-03
 -1.575e-01  6.815e-01  4.100e-03  2.000e-04]
[-0.3249 -0.1479 -0.1966 -0.0859  0.0531  0.2419  0.3975 -0.1356  0.4975
  0.1796 -0.5466  0.4867 -0.3905  0.2787  0.5597  0.2295  0.015  -0.4915
 -0.2677 -0.5103 -0.2086  0.167  -0.1602 -0.369  -0.0104  0.465  -0.501
  0.2909  0.0404 -0.1108 -0.3361  0.5186  0.0912  0.0277]
A,B:  0  out of  34
B,A:  0  out of  34


In [11]:
print('activation: ', model.layers[-1].activation)
print('bias: ', model.layers[-1].bias)
print('weights: ', model.layers[-1].weights)

activation:  ActivationFunction.COSINE
bias:  [-0.87284109]
weights:  [[-0.72488437]
 [-0.85557772]
 [ 0.55538627]
 [-0.0838537 ]]


In [12]:
all_fitness = model_history.historical_particle_fitness()
all_locations = model_history.historical_particle_location()

In [13]:
len(model_history.vec_history[::10])

2320

In [14]:
len(model_history.particle_location[0][::10])

232

In [15]:
# Cubic Most fit model: 4436
Cubic_ann_params = np.array([ 4.06948631e+00, -7.29125756e-01,  5.73721712e-01, -4.97778569e-01, 1.21867021e-01, -4.07830431e-01,  9.23118308e-01,  9.96763804e-01, -3.53176479e-01,  2.82162360e+00, -6.77411971e-01, -6.33183382e-01, 3.17935306e-01, -1.91733120e-01,  2.57710600e-03])

#Linear Most fit model: 9.17486598344206e+25
Linear_ann_params = np.array([0.33653903, 0.62395429, 0.38036724,-0.73197691, 0.27957089, -0.44447831, 0.90125204, -0.72400064, -0.73325263, 0.2053941, -0.37270846, 0.1568939, 0.4157871, -0.37861424, -0.57400409])

#Sine Most fit model: 329
Sine_ann_params = np.array([ 3.50680037,  0.2952162 ,  0.71664711, -0.95665899, -0.9055195, -0.27101866,  0.41973435,  0.36889121, -0.28896977,  3.3595307, 0.42623047,  0.30804742,  0.62065839, -0.73504326,  0.70304275])

#Tanh Most fit model: 20498
Tanh_ann_params = np.array([ 4.30993349,  0.2291253 , -0.99626464, -0.29776704,  0.99032436, 0.54519926, 0.98178641, -0.06046016, 0.9675356 ,  2.17890815, -0.49269196, -0.29330431, -0.94045692, 0.72556016, 0.95398864])



In [16]:

model.decode_vec(Tanh_ann_params)
model.one_pass()

In [19]:
#score function
correctly_classified = 0


#get predictions
#predictions = np.round(model.y_hat, 2)
predictions = model.y_hat
print(model.y)
print(predictions)

#loop through predictions
for index, pred in enumerate(predictions):

    if np.isclose(y_test[index], pred):
        correctly_classified += 1

print('A,B: ', correctly_classified , " out of ", len(y_test))
correctly_classified = 0
#loop through predictions
for index, pred in enumerate(predictions):
    if np.isclose(pred, y_test[index]):
        correctly_classified += 1

print('B,A: ',correctly_classified , " out of ", len(y_test))

[-1.406e-01 -1.060e-02 -2.700e-02 -1.700e-03  1.000e-03  6.400e-02
  2.875e-01 -8.000e-03  5.927e-01  2.700e-02 -8.847e-01  5.514e-01
 -2.621e-01  9.730e-02  8.847e-01  5.490e-02  1.000e-04 -5.927e-01
 -7.410e-02 -6.815e-01 -3.280e-02  2.200e-02 -1.380e-02 -2.160e-01
  0.000e+00  4.746e-01 -6.361e-01  1.106e-01  5.000e-04 -4.100e-03
 -1.575e-01  6.815e-01  4.100e-03  2.000e-04]
[-0.14812342 -0.01279378 -0.03346702  0.00358921  0.02134806  0.07213824
  0.26918008 -0.00879245  0.6025394   0.04466714 -0.86920954  0.55571161
 -0.26711373  0.09792605  0.89808438  0.06528512  0.01742959 -0.60303605
 -0.0829848  -0.69098723 -0.03999464  0.04089869 -0.0172251  -0.22159733
  0.0147349   0.46897127 -0.64643788  0.10854036  0.0200163  -0.00194235
 -0.16449365  0.70143118  0.02589845  0.01872086]
A,B:  0  out of  34
B,A:  0  out of  34


In [18]:
1/model.loss

6847.058729162223